# Creating ETL Pipeline for NoSQL Database

## Part I. ETL Pipeline for Pre-Processing the Files

Files representing data from each day need to be loaded into Cassandra tables designed to handle specific queries. Our company, Sparkify, is interested in being able to get 1) artist, song title, and song length given a session id and a session number, 2) artist, song, and user name for a particular user and session, and 3) every user who has listened to a particular song.

In part 1, I focus on processing the daily csv files to create a single csv for loading into the database.

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
print(full_data_rows_list[0])

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056
['Muse', 'Logged In', 'Harper', 'M', '1', 'Barrett', '209.50159', 'paid', 'New York-Newark-Jersey City, NY-NJ-PA', 'PUT', 'NextSong', '1.54069E+12', '275', 'Supermassive Black Hole (Twilight Soundtrack Version)', '200', '1.54172E+12', '42']


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Create the NoSQL database to store the data

## Now, I will use the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Next, I'll write the code to create an Apache Cassandra database, create tables, and write the queries for each table.

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
KEYSPACE = 'udacity'

In [7]:
session.execute("""
        CREATE KEYSPACE IF NOT EXISTS udacity
        WITH replication = { 'class': 'SimpleStrategy', 'replication_factor': '1' }
        """)

#### Set Keyspace

In [8]:
session.set_keyspace(KEYSPACE)

### I generally explained the queries above. Here the specific query requirements are laid. The query requirements need to specify how the data needs to be partitioned and ordered. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## 1.1 Create the table and the query to access the data

- Columns needed: sessionId, itemInSession, artist, song title song length
- Primary key: session_id and item_in_session: are query requires the ability to filter on these columns, so both need to be in the primary key.

In [9]:
session.execute("""CREATE TABLE IF NOT EXISTS session_and_item_library (
    session_id int,
    item_in_session int,
    artist text,
    song_title text,
    song_length float,
    PRIMARY KEY (session_id, item_in_session)
)""")

table1_query = "SELECT artist, song_title, song_length FROM session_and_item_library WHERE session_id=338 AND item_in_session=4"                   

## 2.1 Create the table and the query to access the data

- Columns needed: userId, sessionId, artist, song, first name, last name
- Primary key: user_id, session_id: the query requires the ability to filter on these columns
- Clustering column: the query requires that he results be ordered by item_in_session, so it is included as a clustering column.

In [10]:
session.execute("""CREATE TABLE IF NOT EXISTS user_session_library (
    user_id int,
    session_id int,
    item_in_session int,
    artist text,
    song_title text,
    first_name text,
    last_name text,
    PRIMARY KEY ((user_id, session_id), item_in_session)
)""")

table2_query = "SELECT artist, song_title, first_name, last_name FROM user_session_library WHERE user_id=10 AND session_id=182"                   

## 3.1 Create the table and the query to access the data

- Columns needed: song, userId, first name, last name
- Primary key: song_title, user_id: the query only asks about filter on song_title, however, we need to include user_id so that we ensure each row is unique

In [11]:
session.execute("""CREATE TABLE IF NOT EXISTS user_song_library (
    song_title text,
    user_id int,
    first_name text,
    last_name text,
    PRIMARY KEY (song_title, user_id)
)""")

table3_query = "SELECT first_name, last_name FROM user_song_library WHERE song_title='All Hands Against His Own'"                                       

## Process the data

In [12]:
# Do all the inserts in a single for loop
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        artist = line[0]
        first_name = line[1]
        gender = line[2]
        item_in_session = int(line[3])
        last_name = line[4]
        song_length = float(line[5])
        level = line[6]
        location = line[7]
        session_id = int(line[8])
        song_title = line[9]
        user_id = int(line[10])
        
        query1 = """INSERT INTO session_and_item_library (session_id, item_in_session, artist, song_title, song_length) """
        query1 = query1 + "VALUES (%s, %s, %s, %s, %s)"
        
        session.execute(query1, (session_id, item_in_session, artist, song_title, song_length))
        
        query2 = """INSERT INTO user_session_library (user_id, session_id, artist, item_in_session, song_title, first_name, last_name) """
        query2 = query2 + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        
        session.execute(query2, (user_id, session_id, artist, item_in_session, song_title, first_name, last_name))
        
        query3 = """INSERT INTO user_song_library (song_title, user_id, first_name, last_name) """
        query3 = query3 + "VALUES (%s, %s, %s, %s)"
        
        session.execute(query3, (song_title, user_id, first_name, last_name))

#### Do a SELECT to verify that the data have been inserted into each table

## 1.2 Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

sessionId indicates a unique session and itemInSession identifies a single song play within a session. This query should produce a single row.

In [13]:
rows = session.execute(table1_query)
for row in rows:
    print (row)

Row(artist='Faithless', song_title='Music Matters (Mark Knight Dub)', song_length=495.30731201171875)


## 2.2 Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

userId and sessionId indicate a listening session by a particular user. The query should return all the songs listened to in a particular session by the user, in the order they listened to them.

In [14]:
rows = session.execute(table2_query)
for row in rows:
    print (row)

Row(artist='Down To The Bone', song_title="Keep On Keepin' On", first_name='Sylvie', last_name='Cruz')
Row(artist='Three Drives', song_title='Greece 2000', first_name='Sylvie', last_name='Cruz')
Row(artist='Sebastien Tellier', song_title='Kilometer', first_name='Sylvie', last_name='Cruz')
Row(artist='Lonnie Gordon', song_title='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', first_name='Sylvie', last_name='Cruz')


## 3.2 Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

The query should return all users who have listened to the song.

In [15]:
rows = session.execute(table3_query)
for row in rows:
    print (row)

Row(first_name='Jacqueline', last_name='Lynch')
Row(first_name='Tegan', last_name='Levine')
Row(first_name='Sara', last_name='Johnson')


### Drop the tables before closing out the sessions

In [16]:
session.execute("""DROP TABLE IF EXISTS session_and_item_library""")
session.execute("""DROP TABLE IF EXISTS user_session_library""")
session.execute("""DROP TABLE IF EXISTS user_song_library""")

### Close the session and cluster connection¶

In [17]:
session.shutdown()
cluster.shutdown()